# Z3 Playground

We use the [Z3 Theorem Prover](https://github.com/z3prover/z3).
Helpful resources are:
- the extensive Python-API-based [tutorial](http://theory.stanford.edu/~nikolaj/programmingz3.html)
- the getting-started [guide](https://rise4fun.com/z3/tutorial/guide) using the Z3 input format
- the Python API documentation in [two](https://z3prover.github.io/api/html/namespacez3py.html) [different](https://z3prover.github.io/api/html/z3.html) formats
- the [Wiki](https://github.com/Z3Prover/z3/wiki) on *GitHub*
- the [online prover](https://rise4fun.com/Z3/)

In [1]:
from z3 import *

# Propositional Logic

Besides solving a simple logical problem, the following example also shows that we can easily handle a whole list of variables.

In [2]:
x = Bools(' '.join(['x' + str(i) for i in range(1, 6)]))
solver = Solver()
solver.add(Not(x[0]))
solver.add(And(x[1:4]))
solver.add(Or(x[2], x[4]))
print('Satisfiable? ' + str(solver.check()))
print('One model: ' + str(solver.model()))

Satisfiable? sat
One model: [x1 = False, x2 = True, x3 = True, x4 = True, x5 = False]
